In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bonavista,CA,2020-12-27 06:48:44,48.65,-53.11,51.80,87,56,12.75
1,1,Lerwick,GB,2020-12-27 06:48:44,60.15,-1.15,41.00,80,100,14.99
2,2,Hermanus,ZA,2020-12-27 06:48:44,-34.42,19.23,66.00,73,97,4.00
3,3,Guadalupe Victoria,MX,2020-12-27 06:48:44,32.29,-115.11,53.60,37,1,2.66
4,4,Tinskoy,RU,2020-12-27 06:48:44,56.15,96.92,-26.21,82,61,3.29


In [3]:
#Check the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Poum,NC,2020-12-27 06:48:45,-20.23,164.02,79.59,76,100,10.63
17,17,Vaini,TO,2020-12-27 06:48:47,-21.20,-175.20,80.60,94,75,9.17
20,20,Fare,PF,2020-12-27 06:48:48,-16.70,-151.02,80.19,78,100,15.66
22,22,Port Alfred,ZA,2020-12-27 06:48:49,-33.59,26.89,75.00,76,48,5.01
23,23,Amaigbo,NG,2020-12-27 06:48:27,5.79,7.84,76.59,72,56,2.21
26,26,Busselton,AU,2020-12-27 06:48:50,-33.65,115.33,79.00,36,0,1.99
38,38,Marawi,PH,2020-12-27 06:48:53,8.00,124.29,81.00,87,100,5.01
40,40,Ewo,CG,2020-12-27 06:48:53,-0.87,14.82,79.16,76,100,6.46
42,42,Geraldton,AU,2020-12-27 06:48:54,-28.77,114.60,84.20,42,0,26.40
45,45,Kununurra,AU,2020-12-27 06:48:55,-15.77,128.73,84.20,79,75,12.75


In [14]:
preferred_cities_df.count()


City_ID       137
City          137
Country       135
Date          137
Lat           137
Lng           137
Max Temp      137
Humidity      137
Cloudiness    137
Wind Speed    137
dtype: int64

In [17]:
preferred_cities_df=preferred_cities_df.dropna() 
preferred_cities_df


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Poum,NC,2020-12-27 06:48:45,-20.23,164.02,79.59,76,100,10.63
17,17,Vaini,TO,2020-12-27 06:48:47,-21.20,-175.20,80.60,94,75,9.17
20,20,Fare,PF,2020-12-27 06:48:48,-16.70,-151.02,80.19,78,100,15.66
22,22,Port Alfred,ZA,2020-12-27 06:48:49,-33.59,26.89,75.00,76,48,5.01
23,23,Amaigbo,NG,2020-12-27 06:48:27,5.79,7.84,76.59,72,56,2.21
...,...,...,...,...,...,...,...,...,...,...
536,536,Upington,ZA,2020-12-27 06:51:10,-28.45,21.26,75.20,36,0,6.93
543,543,Mackay,AU,2020-12-27 06:51:12,-21.15,149.20,84.20,66,3,14.99
544,544,Bagotville,AU,2020-12-27 06:51:12,-28.98,153.42,80.60,65,91,19.46
553,553,Maroantsetra,MG,2020-12-27 06:51:14,-15.43,49.73,87.44,59,100,2.08


In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Poum,NC,79.59,-20.23,164.02,
17,Vaini,TO,80.60,-21.20,-175.20,
20,Fare,PF,80.19,-16.70,-151.02,
22,Port Alfred,ZA,75.00,-33.59,26.89,
23,Amaigbo,NG,76.59,5.79,7.84,
26,Busselton,AU,79.00,-33.65,115.33,
38,Marawi,PH,81.00,8.00,124.29,
40,Ewo,CG,79.16,-0.87,14.82,
42,Geraldton,AU,84.20,-28.77,114.60,
45,Kununurra,AU,84.20,-15.77,128.73,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
 hotel_df
    

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Poum,NC,79.59,-20.23,164.02,Gîte kajeon
17,Vaini,TO,80.60,-21.20,-175.20,Keleti Beach Resort
20,Fare,PF,80.19,-16.70,-151.02,Maitai Lapita Village Huahine
22,Port Alfred,ZA,75.00,-33.59,26.89,The Halyards Hotel
23,Amaigbo,NG,76.59,5.79,7.84,Ikemba Guest House
...,...,...,...,...,...,...
536,Upington,ZA,75.20,-28.45,21.26,River Place Manor
543,Mackay,AU,84.20,-21.15,149.20,International Lodge Motel
544,Bagotville,AU,80.60,-28.98,153.42,Broadwater Sunrise Caravan Park
553,Maroantsetra,MG,87.44,-15.43,49.73,La Pagode


In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))